# 09 - Deteccao de Anomalias: Treino
## Sprint 2 - Dia 10

**Objetivo:** Treinar detector de anomalias com Isolation Forest

**Hipotese H6:** Isolation Forest detecta anomalias com Precision >0.85 e Recall >0.80

**Restricao:** Modelo global (nao por categoria)

**Entregaveis:**
- Modelo `isolation_forest.pkl` treinado
- Predicoes de anomalias para todo o dataset

## 1. Setup e Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import pickle
import os

# Configuracoes
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

In [ ]:
# Carregar transacoes
transacoes = pd.read_csv('../data/raw/transacoes.csv')

print(f"Total de transacoes: {len(transacoes)}")
print(f"\nColunas: {list(transacoes.columns)}")
print(f"\nDistribuicao de anomalias (ground truth):")
print(transacoes['is_anomalia'].value_counts())
print(f"\nPercentual de anomalias: {transacoes['is_anomalia'].mean()*100:.2f}%")

In [ ]:
# Filtrar apenas gastos (excluir Renda)
gastos = transacoes[transacoes['categoria'] != 'Renda'].copy()

print(f"Transacoes de gasto: {len(gastos)}")
print(f"Anomalias em gastos: {gastos['is_anomalia'].sum()}")
print(f"Percentual: {gastos['is_anomalia'].mean()*100:.2f}%")

## 2. Analise Exploratoria das Anomalias

In [ ]:
# Comparar valores normais vs anomalias
print("="*70)
print("COMPARACAO: TRANSACOES NORMAIS vs ANOMALIAS")
print("="*70)

normais = gastos[gastos['is_anomalia'] == False]['valor']
anomalias = gastos[gastos['is_anomalia'] == True]['valor']

print(f"\nTransacoes Normais:")
print(f"  Count: {len(normais)}")
print(f"  Media: R$ {normais.mean():.2f}")
print(f"  Mediana: R$ {normais.median():.2f}")
print(f"  Std: R$ {normais.std():.2f}")
print(f"  Max: R$ {normais.max():.2f}")

print(f"\nTransacoes Anomalas:")
print(f"  Count: {len(anomalias)}")
print(f"  Media: R$ {anomalias.mean():.2f}")
print(f"  Mediana: R$ {anomalias.median():.2f}")
print(f"  Std: R$ {anomalias.std():.2f}")
print(f"  Max: R$ {anomalias.max():.2f}")

print(f"\nRatio (anomalia/normal): {anomalias.mean()/normais.mean():.2f}x")

In [ ]:
# Visualizar distribuicao de valores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(normais, bins=50, alpha=0.7, label='Normal', color='blue')
axes[0].hist(anomalias, bins=50, alpha=0.7, label='Anomalia', color='red')
axes[0].set_xlabel('Valor (R$)')
axes[0].set_ylabel('Frequencia')
axes[0].set_title('Distribuicao de Valores: Normal vs Anomalia')
axes[0].legend()
axes[0].set_xlim(0, 2000)  # Limitar para melhor visualizacao

# Boxplot
gastos['tipo'] = gastos['is_anomalia'].map({True: 'Anomalia', False: 'Normal'})
sns.boxplot(data=gastos, x='tipo', y='valor', ax=axes[1], palette=['blue', 'red'])
axes[1].set_ylabel('Valor (R$)')
axes[1].set_title('Boxplot: Normal vs Anomalia')

plt.tight_layout()
plt.savefig('../outputs/anomalias_distribuicao.png', dpi=150, bbox_inches='tight')
plt.show()
print("Grafico salvo: outputs/anomalias_distribuicao.png")

In [ ]:
# Anomalias por categoria
print("\n" + "="*70)
print("ANOMALIAS POR CATEGORIA")
print("="*70)

anomalias_por_cat = gastos.groupby('categoria').agg({
    'is_anomalia': ['sum', 'mean', 'count']
}).round(4)
anomalias_por_cat.columns = ['num_anomalias', 'pct_anomalias', 'total_transacoes']
anomalias_por_cat['pct_anomalias'] = anomalias_por_cat['pct_anomalias'] * 100
anomalias_por_cat = anomalias_por_cat.sort_values('num_anomalias', ascending=False)

print("\n")
print(anomalias_por_cat.to_string())

## 3. Feature Engineering para Deteccao de Anomalias

In [ ]:
# Calcular estatisticas por categoria para normalizar valores
stats_categoria = gastos.groupby('categoria')['valor'].agg(['mean', 'std', 'median']).reset_index()
stats_categoria.columns = ['categoria', 'media_cat', 'std_cat', 'mediana_cat']

print("Estatisticas por categoria:")
print(stats_categoria.to_string())

In [ ]:
# Merge estatisticas com gastos
gastos_features = gastos.merge(stats_categoria, on='categoria')

# Criar features para o modelo
# Feature 1: Valor absoluto
gastos_features['feat_valor'] = gastos_features['valor']

# Feature 2: Valor normalizado pela media da categoria (z-score simplificado)
gastos_features['feat_valor_norm'] = (
    (gastos_features['valor'] - gastos_features['media_cat']) / gastos_features['std_cat']
)

# Feature 3: Ratio valor/mediana da categoria
gastos_features['feat_ratio_mediana'] = gastos_features['valor'] / gastos_features['mediana_cat']

# Feature 4: Log do valor (para lidar com skewness)
gastos_features['feat_log_valor'] = np.log1p(gastos_features['valor'])

print("Features criadas:")
print("  - feat_valor: Valor absoluto")
print("  - feat_valor_norm: Z-score por categoria")
print("  - feat_ratio_mediana: Ratio valor/mediana da categoria")
print("  - feat_log_valor: Log do valor")

gastos_features[['valor', 'categoria', 'feat_valor', 'feat_valor_norm', 'feat_ratio_mediana', 'feat_log_valor', 'is_anomalia']].head(10)

In [ ]:
# Verificar correlacao das features com anomalias
print("\n" + "="*70)
print("CORRELACAO DAS FEATURES COM ANOMALIAS")
print("="*70)

features_cols = ['feat_valor', 'feat_valor_norm', 'feat_ratio_mediana', 'feat_log_valor']

for feat in features_cols:
    corr = gastos_features[feat].corr(gastos_features['is_anomalia'].astype(int))
    print(f"  {feat}: {corr:.4f}")

## 4. Preparacao dos Dados para Treino

In [ ]:
# Selecionar features para o modelo
# Usando apenas feat_valor_norm e feat_ratio_mediana (mais discriminativas)
FEATURES_MODELO = ['feat_valor_norm', 'feat_ratio_mediana']

X = gastos_features[FEATURES_MODELO].copy()
y_true = gastos_features['is_anomalia'].astype(int).values

# Tratar valores infinitos ou NaN
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

print(f"Shape dos dados: {X.shape}")
print(f"Features: {FEATURES_MODELO}")
print(f"\nEstatisticas das features:")
print(X.describe())

In [ ]:
# Normalizar features com StandardScaler
scaler_anomalias = StandardScaler()
X_scaled = scaler_anomalias.fit_transform(X)

print("Features normalizadas com StandardScaler")
print(f"Shape: {X_scaled.shape}")

## 5. Treinar Isolation Forest

In [ ]:
# Parametros do modelo
# contamination = proporcao esperada de anomalias (5% no dataset)
CONTAMINATION = 0.05

print("="*70)
print("TREINAMENTO DO ISOLATION FOREST")
print("="*70)
print(f"\nParametros:")
print(f"  - contamination: {CONTAMINATION}")
print(f"  - n_estimators: 100 (default)")
print(f"  - random_state: 42")

In [ ]:
# Treinar modelo
isolation_forest = IsolationForest(
    contamination=CONTAMINATION,
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

isolation_forest.fit(X_scaled)

print("\nModelo treinado com sucesso!")

In [ ]:
# Gerar predicoes
# Isolation Forest retorna: 1 para normal, -1 para anomalia
y_pred_raw = isolation_forest.predict(X_scaled)

# Converter para 0/1 (0=normal, 1=anomalia)
y_pred = (y_pred_raw == -1).astype(int)

print(f"\nPredicoes geradas:")
print(f"  Normais detectados: {(y_pred == 0).sum()}")
print(f"  Anomalias detectadas: {(y_pred == 1).sum()}")
print(f"  Percentual anomalias: {y_pred.mean()*100:.2f}%")

In [ ]:
# Obter scores de anomalia (quanto menor, mais anomalo)
anomaly_scores = isolation_forest.decision_function(X_scaled)

# Adicionar ao dataframe
gastos_features['anomaly_score'] = anomaly_scores
gastos_features['pred_anomalia'] = y_pred

print("Scores de anomalia calculados")
print(f"\nScore medio - Normais (ground truth): {gastos_features[gastos_features['is_anomalia']==False]['anomaly_score'].mean():.4f}")
print(f"Score medio - Anomalias (ground truth): {gastos_features[gastos_features['is_anomalia']==True]['anomaly_score'].mean():.4f}")

## 6. Avaliacao Preliminar

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

# Calcular metricas
print("="*70)
print("AVALIACAO PRELIMINAR")
print("="*70)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"\n>>> PRECISION: {precision:.4f} (target: >0.85)")
print(f">>> RECALL: {recall:.4f} (target: >0.80)")
print(f">>> F1-SCORE: {f1:.4f}")

# Status
precision_ok = precision >= 0.85
recall_ok = recall >= 0.80

print(f"\nStatus Precision: {'ATINGIDO' if precision_ok else 'NAO ATINGIDO'}")
print(f"Status Recall: {'ATINGIDO' if recall_ok else 'NAO ATINGIDO'}")

In [ ]:
# Matriz de confusao
cm = confusion_matrix(y_true, y_pred)

print("\nMatriz de Confusao:")
print(f"                 Pred Normal  Pred Anomalia")
print(f"Real Normal      {cm[0,0]:>10}  {cm[0,1]:>13}")
print(f"Real Anomalia    {cm[1,0]:>10}  {cm[1,1]:>13}")

print(f"\nInterpretacao:")
print(f"  True Negatives (TN): {cm[0,0]} - Normais corretamente identificados")
print(f"  False Positives (FP): {cm[0,1]} - Normais classificados como anomalia")
print(f"  False Negatives (FN): {cm[1,0]} - Anomalias nao detectadas")
print(f"  True Positives (TP): {cm[1,1]} - Anomalias corretamente detectadas")

In [ ]:
# Visualizar matriz de confusao
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Normal', 'Anomalia'],
            yticklabels=['Normal', 'Anomalia'])
ax.set_xlabel('Predicao')
ax.set_ylabel('Real (Ground Truth)')
ax.set_title(f'Matriz de Confusao - Isolation Forest\nPrecision: {precision:.2%} | Recall: {recall:.2%}')

plt.tight_layout()
plt.savefig('../outputs/matriz_confusao_anomalias.png', dpi=150, bbox_inches='tight')
plt.show()
print("Grafico salvo: outputs/matriz_confusao_anomalias.png")

In [ ]:
# Visualizar distribuicao dos scores
fig, ax = plt.subplots(figsize=(10, 6))

scores_normais = gastos_features[gastos_features['is_anomalia']==False]['anomaly_score']
scores_anomalias = gastos_features[gastos_features['is_anomalia']==True]['anomaly_score']

ax.hist(scores_normais, bins=50, alpha=0.7, label='Normal (ground truth)', color='blue')
ax.hist(scores_anomalias, bins=50, alpha=0.7, label='Anomalia (ground truth)', color='red')
ax.axvline(x=0, color='green', linestyle='--', label='Threshold (score=0)')
ax.set_xlabel('Anomaly Score (menor = mais anomalo)')
ax.set_ylabel('Frequencia')
ax.set_title('Distribuicao dos Scores de Anomalia')
ax.legend()

plt.tight_layout()
plt.savefig('../outputs/distribuicao_scores_anomalia.png', dpi=150, bbox_inches='tight')
plt.show()
print("Grafico salvo: outputs/distribuicao_scores_anomalia.png")

## 7. Salvar Modelo e Artefatos

In [ ]:
# Criar diretorio models se nao existir
os.makedirs('../models', exist_ok=True)

# Salvar modelo Isolation Forest
with open('../models/isolation_forest.pkl', 'wb') as f:
    pickle.dump(isolation_forest, f)
print("Modelo salvo: models/isolation_forest.pkl")

# Salvar scaler
with open('../models/scaler_anomalias.pkl', 'wb') as f:
    pickle.dump(scaler_anomalias, f)
print("Scaler salvo: models/scaler_anomalias.pkl")

# Salvar estatisticas por categoria (necessario para novas predicoes)
stats_categoria.to_csv('../models/stats_categoria_anomalias.csv', index=False)
print("Estatisticas salvas: models/stats_categoria_anomalias.csv")

In [ ]:
# Salvar transacoes com predicoes
colunas_saida = [
    'user_id', 'data', 'categoria', 'valor', 'mes', 'ano',
    'is_essencial', 'is_anomalia', 'anomaly_score', 'pred_anomalia'
]

transacoes_com_pred = gastos_features[colunas_saida].copy()
transacoes_com_pred.to_csv('../data/processed/transacoes_com_anomalias_pred.csv', index=False)

print(f"\nTransacoes com predicoes salvas: data/processed/transacoes_com_anomalias_pred.csv")
print(f"Total de linhas: {len(transacoes_com_pred)}")

In [ ]:
# Salvar configuracao do modelo
import json

config_modelo = {
    'versao': '1.0',
    'data_treino': '2026-01-26',
    'modelo': 'IsolationForest',
    'parametros': {
        'contamination': CONTAMINATION,
        'n_estimators': 100,
        'random_state': 42
    },
    'features': FEATURES_MODELO,
    'metricas_treino': {
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'f1_score': round(f1, 4)
    },
    'targets': {
        'precision': 0.85,
        'recall': 0.80
    },
    'arquivos': {
        'modelo': 'isolation_forest.pkl',
        'scaler': 'scaler_anomalias.pkl',
        'stats_categoria': 'stats_categoria_anomalias.csv'
    }
}

with open('../models/config_anomalias.json', 'w', encoding='utf-8') as f:
    json.dump(config_modelo, f, indent=2, ensure_ascii=False)

print("Configuracao salva: models/config_anomalias.json")

## 8. Funcao de Predicao para Novos Dados

In [ ]:
def detectar_anomalia(transacao, modelo, scaler, stats_cat):
    """
    Detecta se uma transacao e anomala.
    
    Parametros:
        transacao: dict com 'valor' e 'categoria'
        modelo: IsolationForest treinado
        scaler: StandardScaler treinado
        stats_cat: DataFrame com estatisticas por categoria
    
    Retorna:
        dict com 'is_anomalia', 'score', 'confianca'
    """
    valor = transacao['valor']
    categoria = transacao['categoria']
    
    # Buscar estatisticas da categoria
    cat_stats = stats_cat[stats_cat['categoria'] == categoria]
    
    if len(cat_stats) == 0:
        # Categoria desconhecida - usar media geral
        media_cat = stats_cat['media_cat'].mean()
        std_cat = stats_cat['std_cat'].mean()
        mediana_cat = stats_cat['mediana_cat'].mean()
    else:
        media_cat = cat_stats['media_cat'].values[0]
        std_cat = cat_stats['std_cat'].values[0]
        mediana_cat = cat_stats['mediana_cat'].values[0]
    
    # Calcular features
    feat_valor_norm = (valor - media_cat) / std_cat if std_cat > 0 else 0
    feat_ratio_mediana = valor / mediana_cat if mediana_cat > 0 else 0
    
    # Preparar input
    X = np.array([[feat_valor_norm, feat_ratio_mediana]])
    X = np.nan_to_num(X, nan=0, posinf=0, neginf=0)
    X_scaled = scaler.transform(X)
    
    # Predicao
    pred = modelo.predict(X_scaled)[0]
    score = modelo.decision_function(X_scaled)[0]
    
    is_anomalia = pred == -1
    
    return {
        'is_anomalia': is_anomalia,
        'score': round(score, 4),
        'confianca': 'alta' if abs(score) > 0.1 else 'media' if abs(score) > 0.05 else 'baixa'
    }

print("Funcao detectar_anomalia() definida!")

In [ ]:
# Testar funcao com exemplos
print("\n" + "="*70)
print("TESTE DA FUNCAO DE DETECCAO")
print("="*70)

exemplos = [
    {'valor': 50.0, 'categoria': 'Alimentacao_Fora', 'descricao': 'Gasto normal'},
    {'valor': 500.0, 'categoria': 'Alimentacao_Fora', 'descricao': 'Gasto alto'},
    {'valor': 1500.0, 'categoria': 'Alimentacao_Fora', 'descricao': 'Gasto muito alto'},
    {'valor': 100.0, 'categoria': 'Vestuario', 'descricao': 'Gasto normal'},
    {'valor': 800.0, 'categoria': 'Vestuario', 'descricao': 'Gasto alto'},
]

for ex in exemplos:
    resultado = detectar_anomalia(ex, isolation_forest, scaler_anomalias, stats_categoria)
    status = "ANOMALIA" if resultado['is_anomalia'] else "NORMAL"
    print(f"\n{ex['descricao']} - {ex['categoria']}: R$ {ex['valor']:.2f}")
    print(f"  -> {status} (score: {resultado['score']}, confianca: {resultado['confianca']})")

## 9. Checklist e Proximos Passos

In [ ]:
# Checklist Dia 10
print("\n" + "="*70)
print("CHECKLIST DIA 10")
print("="*70)
print("\n[x] Carregar e analisar transacoes")
print("[x] Feature engineering para anomalias")
print("[x] Treinar Isolation Forest global")
print("[x] Gerar predicoes para todo o dataset")
print("[x] Avaliacao preliminar (precision, recall)")
print("[x] Salvar modelo e artefatos")
print("[x] Criar funcao de predicao para novos dados")
print("\n>>> DIA 10 CONCLUIDO!")
print("\nProximo passo (Dia 11): Validacao detalhada da H6 e ajustes")

In [ ]:
# Resumo final
print("\n" + "="*70)
print("RESUMO FINAL - DIA 10")
print("="*70)
print(f"\nModelo: Isolation Forest")
print(f"Features: {FEATURES_MODELO}")
print(f"Contamination: {CONTAMINATION}")
print(f"\nMetricas preliminares:")
print(f"  Precision: {precision:.4f} (target: >0.85) - {'OK' if precision_ok else 'AJUSTAR'}")
print(f"  Recall: {recall:.4f} (target: >0.80) - {'OK' if recall_ok else 'AJUSTAR'}")
print(f"  F1-Score: {f1:.4f}")
print(f"\nArquivos gerados:")
print("  - models/isolation_forest.pkl")
print("  - models/scaler_anomalias.pkl")
print("  - models/stats_categoria_anomalias.csv")
print("  - models/config_anomalias.json")
print("  - data/processed/transacoes_com_anomalias_pred.csv")
print("  - outputs/anomalias_distribuicao.png")
print("  - outputs/matriz_confusao_anomalias.png")
print("  - outputs/distribuicao_scores_anomalia.png")